# 공공 데이터 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 만개의 레시피 공공 데이터 불러오기
recipe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/크롤링/recipe.csv', encoding='cp949')

In [ ]:
# 인덱스 설정
recipe.set_index('id', inplace = True)

In [ ]:
# 추가 정보 열 추가
recipe['thumbanil'] = ''
recipe['time'] = ''
recipe['orders'] = ''
recipe['photo'] = ''

In [ ]:
# 레시피 id 리스트
id_list = recipe.index.to_list()

In [ ]:
recipe

,name,composition,ingredients,servings,difficulty,thumbanil,time,orders,photo
id,,,,,,,,,
6842324,고구마맛탕,가볍게,[재료] 고구마| 식용유| 황설탕| 올리고당| 견과류| 물,2인분,초급,,,,
6845721,참치샐러드비빔밥,든든하게,[재료] 참치 통조림 1캔| 쌈채소 1줌| 밥 1공기| 초고추장 적당히 [선택재료]...,1인분,아무나,,,,
6845906,해쉬브라운그라탕,든든하게,[필수재료] 해쉬브라운(냉동감자) 4장| 올리브유 적당량| 토마토소스 2/3컵| 슈...,2인분,초급,,,,
6846020,식빵핫도그,가볍게,[필수재료] 식빵 3장| 소시지 3개| 슬라이스치즈 3장| 달걀 2개| 빵가루 1컵...,1인분,초급,,,,
6846262,딸기컵케이크,가볍게,[필수재료] 머핀 2개| 딸기 8개| 플레인요구르트 1컵 [선택재료] 판 초콜릿 적...,2인분,아무나,,,,
...,...,...,...,...,...,...,...,...,...
6952225,유자생강잼,가볍게,[재료] 유자 5개| 설탕 250g| 생강 1쪽,4인분,초급,,,,
6952295,진저케이크,가볍게,[재료] 무염버터 110g| 흑설탕 90g| 꿀 30g| 달걀 50g| 중력분 15...,4인분,아무나,,,,
6952314,곶감잼,가볍게,[재료] 곶감 300g| 황설탕 300g| 생크림 250g,4인분,초급,,,,


# 공공 데이터에 크롤링 정보 추가하기

In [ ]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
!pip install pyOpenSSL
import ssl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# 레시피 id로 레시피 추가 정보 크롤링
def RecipeCrawler(recipeId):
    
    url = f'https://www.10000recipe.com/recipe/{recipeId}'

    response = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, verify=False)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 레시피 썸네일
    try:
        thumbnail = soup.select_one('#contents_area > div.view2_pic > div.centeredcrop img')['src']
        recipe.loc[recipeId,'thumbanil'] = thumbnail
    except TypeError:
        recipe.loc[recipeId,'thumbanil'] = ''
    pass

    # 레시피 시간
    try:
        time = soup.select_one('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2').get_text()
        recipe.loc[recipeId,'time'] = time
    except AttributeError:
        recipe.loc[recipeId,'time'] = ''
    pass

    # 레시피 조리 순서
    try:
        elements = soup.find_all('div', 'view_step_cont')
        tmp = ''
        for i, element in enumerate(elements, start=1):
            order = element.find('div', 'media-body')
            order = ''.join(order.findAll(string=True, recursive=False)).strip() # 조리 순서에서 설명, 주방 도구 제외
            tmp = tmp + ('|' + str(i) + '. ' + order)
        recipe.loc[recipeId,'orders'] = tmp
    except AttributeError:
        recipe.loc[recipeId,'orders'] = ''
    pass

    # 레시피 조리 사진
    try:
        elements = soup.find_all('div', 'view_step_cont')
        tmp = ''
        for i, element in enumerate(elements, start=1):
            try:
                photo = element.find('img')['src']
                tmp = tmp + ('|' + photo)
            except TypeError:
                tmp = tmp + ('|' + ' ')
            pass
        recipe.loc[recipeId,'photo'] = tmp
    except AttributeError:
        recipe.loc[recipeId,'photo'] = ''
    pass

In [ ]:
for i, id in enumerate(id_list, start=1):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

1번째 크롤링 id: 6842324
2번째 크롤링 id: 6845721
3번째 크롤링 id: 6845906
4번째 크롤링 id: 6846020
5번째 크롤링 id: 6846262
6번째 크롤링 id: 6846342
7번째 크롤링 id: 6846428
8번째 크롤링 id: 6846512
9번째 크롤링 id: 6846633
10번째 크롤링 id: 6846770
11번째 크롤링 id: 6846922
12번째 크롤링 id: 6847060
13번째 크롤링 id: 6847207
14번째 크롤링 id: 6847328
15번째 크롤링 id: 6847465
16번째 크롤링 id: 6847539
17번째 크롤링 id: 6847599
18번째 크롤링 id: 6847602
19번째 크롤링 id: 6847608
20번째 크롤링 id: 6847802
21번째 크롤링 id: 6847891
22번째 크롤링 id: 6847965
23번째 크롤링 id: 6848026
24번째 크롤링 id: 6848085
25번째 크롤링 id: 6848087
26번째 크롤링 id: 6848090
27번째 크롤링 id: 6848243
28번째 크롤링 id: 6848305
29번째 크롤링 id: 6848367
30번째 크롤링 id: 6848372
31번째 크롤링 id: 6848374
32번째 크롤링 id: 6848383
33번째 크롤링 id: 6848386
34번째 크롤링 id: 6849775
35번째 크롤링 id: 6849858
36번째 크롤링 id: 6849910
37번째 크롤링 id: 6849980
38번째 크롤링 id: 6850019
39번째 크롤링 id: 6850026
40번째 크롤링 id: 6850027
41번째 크롤링 id: 6850028
42번째 크롤링 id: 6850035
43번째 크롤링 id: 6850169
44번째 크롤링 id: 6850247
45번째 크롤링 id: 6850322
46번째 크롤링 id: 6850358
47번째 크롤링 id: 6850368
48번째 크롤링 id: 6850369
4

ConnectionError: ignored

In [ ]:
# 끊긴 레시피부터 다시
for i, id in enumerate(id_list[832:], start=833):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

833번째 크롤링 id: 6877023
834번째 크롤링 id: 6877036
835번째 크롤링 id: 6877093
836번째 크롤링 id: 6877116
837번째 크롤링 id: 6877117
838번째 크롤링 id: 6877157
839번째 크롤링 id: 6877173
840번째 크롤링 id: 6877207
841번째 크롤링 id: 6877211
842번째 크롤링 id: 6877212
843번째 크롤링 id: 6877220
844번째 크롤링 id: 6877221
845번째 크롤링 id: 6877228
846번째 크롤링 id: 6877269
847번째 크롤링 id: 6877275
848번째 크롤링 id: 6877282
849번째 크롤링 id: 6877283
850번째 크롤링 id: 6877605
851번째 크롤링 id: 6877613
852번째 크롤링 id: 6877614
853번째 크롤링 id: 6877671
854번째 크롤링 id: 6877672
855번째 크롤링 id: 6877735
856번째 크롤링 id: 6877739
857번째 크롤링 id: 6877741
858번째 크롤링 id: 6877742
859번째 크롤링 id: 6877798
860번째 크롤링 id: 6877799
861번째 크롤링 id: 6877811
862번째 크롤링 id: 6877957
863번째 크롤링 id: 6877959
864번째 크롤링 id: 6878035
865번째 크롤링 id: 6878055
866번째 크롤링 id: 6878106
867번째 크롤링 id: 6878109
868번째 크롤링 id: 6878178
869번째 크롤링 id: 6878179
870번째 크롤링 id: 6878180
871번째 크롤링 id: 6878181
872번째 크롤링 id: 6878230
873번째 크롤링 id: 6878232
874번째 크롤링 id: 6878233
875번째 크롤링 id: 6878234
876번째 크롤링 id: 6878261
877번째 크롤링 id: 6878346
878번째 크롤링 

ConnectionError: ignored

In [ ]:
# 끊긴 레시피부터 다시
for i, id in enumerate(id_list[1677:], start=1678):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

1678번째 크롤링 id: 6894669
1679번째 크롤링 id: 6894679
1680번째 크롤링 id: 6894681
1681번째 크롤링 id: 6894682
1682번째 크롤링 id: 6894683
1683번째 크롤링 id: 6894685
1684번째 크롤링 id: 6894686
1685번째 크롤링 id: 6894838
1686번째 크롤링 id: 6894839
1687번째 크롤링 id: 6894886
1688번째 크롤링 id: 6894903
1689번째 크롤링 id: 6894904
1690번째 크롤링 id: 6894933
1691번째 크롤링 id: 6894935
1692번째 크롤링 id: 6894943
1693번째 크롤링 id: 6894945
1694번째 크롤링 id: 6894948
1695번째 크롤링 id: 6894949
1696번째 크롤링 id: 6894986
1697번째 크롤링 id: 6894987
1698번째 크롤링 id: 6895026
1699번째 크롤링 id: 6895032
1700번째 크롤링 id: 6895036
1701번째 크롤링 id: 6895038
1702번째 크롤링 id: 6895046
1703번째 크롤링 id: 6895047
1704번째 크롤링 id: 6895167
1705번째 크롤링 id: 6895168
1706번째 크롤링 id: 6895170
1707번째 크롤링 id: 6895171
1708번째 크롤링 id: 6895178
1709번째 크롤링 id: 6895242
1710번째 크롤링 id: 6895244
1711번째 크롤링 id: 6895245
1712번째 크롤링 id: 6895293
1713번째 크롤링 id: 6895370
1714번째 크롤링 id: 6895373
1715번째 크롤링 id: 6895374
1716번째 크롤링 id: 6895421
1717번째 크롤링 id: 6895437
1718번째 크롤링 id: 6895438
1719번째 크롤링 id: 6895439
1720번째 크롤링 id: 6895542
1721번째 크롤링 

SSLError: ignored

In [ ]:
# 끊긴 레시피부터 다시
for i, id in enumerate(id_list[2212:], start=2213):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

2213번째 크롤링 id: 6907199
2214번째 크롤링 id: 6907201
2215번째 크롤링 id: 6907206
2216번째 크롤링 id: 6907263
2217번째 크롤링 id: 6907277
2218번째 크롤링 id: 6907301
2219번째 크롤링 id: 6907385
2220번째 크롤링 id: 6907392
2221번째 크롤링 id: 6907397
2222번째 크롤링 id: 6907408
2223번째 크롤링 id: 6907450
2224번째 크롤링 id: 6907461
2225번째 크롤링 id: 6907470
2226번째 크롤링 id: 6907471
2227번째 크롤링 id: 6907515
2228번째 크롤링 id: 6907531
2229번째 크롤링 id: 6907758
2230번째 크롤링 id: 6907759
2231번째 크롤링 id: 6907760
2232번째 크롤링 id: 6907761
2233번째 크롤링 id: 6907796
2234번째 크롤링 id: 6907802
2235번째 크롤링 id: 6907833
2236번째 크롤링 id: 6907890
2237번째 크롤링 id: 6907902
2238번째 크롤링 id: 6907948
2239번째 크롤링 id: 6907965
2240번째 크롤링 id: 6908019
2241번째 크롤링 id: 6908024
2242번째 크롤링 id: 6908126
2243번째 크롤링 id: 6908159
2244번째 크롤링 id: 6908165
2245번째 크롤링 id: 6908203
2246번째 크롤링 id: 6908216
2247번째 크롤링 id: 6908278
2248번째 크롤링 id: 6908286
2249번째 크롤링 id: 6908294
2250번째 크롤링 id: 6908314
2251번째 크롤링 id: 6908359
2252번째 크롤링 id: 6908367
2253번째 크롤링 id: 6908379
2254번째 크롤링 id: 6908435
2255번째 크롤링 id: 6908451
2256번째 크롤링 

ConnectionError: ignored

In [ ]:
# 끊긴 레시피부터 다시
for i, id in enumerate(id_list[2907:], start=2908):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

2908번째 크롤링 id: 6920662
2909번째 크롤링 id: 6920706
2910번째 크롤링 id: 6920727
2911번째 크롤링 id: 6920761
2912번째 크롤링 id: 6920775
2913번째 크롤링 id: 6920790
2914번째 크롤링 id: 6920824
2915번째 크롤링 id: 6920838
2916번째 크롤링 id: 6920871
2917번째 크롤링 id: 6920896
2918번째 크롤링 id: 6920913
2919번째 크롤링 id: 6920933
2920번째 크롤링 id: 6920973
2921번째 크롤링 id: 6920979
2922번째 크롤링 id: 6920994
2923번째 크롤링 id: 6921009
2924번째 크롤링 id: 6921045
2925번째 크롤링 id: 6921056
2926번째 크롤링 id: 6921064
2927번째 크롤링 id: 6921092
2928번째 크롤링 id: 6921099
2929번째 크롤링 id: 6921100
2930번째 크롤링 id: 6921106
2931번째 크롤링 id: 6921144
2932번째 크롤링 id: 6921161
2933번째 크롤링 id: 6921190
2934번째 크롤링 id: 6921201
2935번째 크롤링 id: 6921214
2936번째 크롤링 id: 6921234
2937번째 크롤링 id: 6921244
2938번째 크롤링 id: 6921256
2939번째 크롤링 id: 6921271
2940번째 크롤링 id: 6921289
2941번째 크롤링 id: 6921314
2942번째 크롤링 id: 6921355
2943번째 크롤링 id: 6921372
2944번째 크롤링 id: 6921374
2945번째 크롤링 id: 6921401
2946번째 크롤링 id: 6921414
2947번째 크롤링 id: 6921420
2948번째 크롤링 id: 6921454
2949번째 크롤링 id: 6921463
2950번째 크롤링 id: 6921470
2951번째 크롤링 

ConnectionError: ignored

In [ ]:
# 끊긴 레시피부터 다시
for i, id in enumerate(id_list[3667:], start=3668):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

3668번째 크롤링 id: 6950004
3669번째 크롤링 id: 6950111
3670번째 크롤링 id: 6950180
3671번째 크롤링 id: 6950279
3672번째 크롤링 id: 6950380
3673번째 크롤링 id: 6950416
3674번째 크롤링 id: 6950453
3675번째 크롤링 id: 6950589
3676번째 크롤링 id: 6950626
3677번째 크롤링 id: 6950696
3678번째 크롤링 id: 6950749
3679번째 크롤링 id: 6950822
3680번째 크롤링 id: 6950920
3681번째 크롤링 id: 6951053
3682번째 크롤링 id: 6951078
3683번째 크롤링 id: 6951153
3684번째 크롤링 id: 6951233
3685번째 크롤링 id: 6951315
3686번째 크롤링 id: 6951362
3687번째 크롤링 id: 6951417
3688번째 크롤링 id: 6951522
3689번째 크롤링 id: 6951639
3690번째 크롤링 id: 6951743
3691번째 크롤링 id: 6951794
3692번째 크롤링 id: 6951842
3693번째 크롤링 id: 6951897
3694번째 크롤링 id: 6951955
3695번째 크롤링 id: 6952034
3696번째 크롤링 id: 6952061
3697번째 크롤링 id: 6952081
3698번째 크롤링 id: 6952093
3699번째 크롤링 id: 6952149
3700번째 크롤링 id: 6952225
3701번째 크롤링 id: 6952295
3702번째 크롤링 id: 6952314
3703번째 크롤링 id: 6952344
3704번째 크롤링 id: 6952391


In [ ]:
recipe

,name,composition,ingredients,servings,difficulty,thumbanil,time,orders,photo
id,,,,,,,,,
6842324,고구마맛탕,가볍게,[재료] 고구마| 식용유| 황설탕| 올리고당| 견과류| 물,2인분,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,60분 이내,|1. 바삭하게 튀기는 꿀팁|2. 달콤한 소스 꿀팁|3. 더 건강하게 먹는 꿀팁,|https://recipe1.ezmember.co.kr/cache/recipe/2...
6845721,참치샐러드비빔밥,든든하게,[재료] 참치 통조림 1캔| 쌈채소 1줌| 밥 1공기| 초고추장 적당히 [선택재료]...,1인분,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,10분 이내,|1. 참치는 캔 뚜껑을 따서 기름기를 따라낸다.|2. 볼에 참치를 넣고 다진 양파...,|https://recipe1.ezmember.co.kr/cache/recipe/2...
6845906,해쉬브라운그라탕,든든하게,[필수재료] 해쉬브라운(냉동감자) 4장| 올리브유 적당량| 토마토소스 2/3컵| 슈...,2인분,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,30분 이내,|1. 뜨겁게 달군 팬에 올리브유(2큰술)를 두르고 채 썬 양파와 다진 마늘(½큰술...,|https://recipe1.ezmember.co.kr/cache/recipe/2...
6846020,식빵핫도그,가볍게,[필수재료] 식빵 3장| 소시지 3개| 슬라이스치즈 3장| 달걀 2개| 빵가루 1컵...,1인분,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,30분 이내,|1. 식빵은 밀대를 이용해 납작하게 민다.|2. 깻잎 – 슬라이스치즈 - 소시지 ...,|https://recipe1.ezmember.co.kr/cache/recipe/2...
6846262,딸기컵케이크,가볍게,[필수재료] 머핀 2개| 딸기 8개| 플레인요구르트 1컵 [선택재료] 판 초콜릿 적...,2인분,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,30분 이내,|1. 머핀은 사방 1.5cm 크기로 썰어요.|2. 판초콜릿은 칼로 잘게 부숴요.|...,|https://recipe1.ezmember.co.kr/cache/recipe/2...
...,...,...,...,...,...,...,...,...,...
6952225,유자생강잼,가볍게,[재료] 유자 5개| 설탕 250g| 생강 1쪽,4인분,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,60분 이내,|1. 유자는 깨끗이 씻어 4등분 해 껍질과 과육을 분리한 후 씨를 제거한다.|2....,|https://recipe1.ezmember.co.kr/cache/recipe/2...
6952295,진저케이크,가볍게,[재료] 무염버터 110g| 흑설탕 90g| 꿀 30g| 달걀 50g| 중력분 15...,4인분,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,60분 이내,"|1. 냄비에 무염버터, 흑설탕, 꿀을 넣고 약불로 버터를 녹이면서 섞는다.|2. ...",|https://recipe1.ezmember.co.kr/cache/recipe/2...
6952314,곶감잼,가볍게,[재료] 곶감 300g| 황설탕 300g| 생크림 250g,4인분,초급,https://recipe1.ezmember.co.kr/cache/recipe/20...,30분 이내,"|1. 곶감은 꼭지, 씨를 제거 한 뒤 숟가락으로 속을 긁어내고 껍질은 다진다.|2...",|https://recipe1.ezmember.co.kr/cache/recipe/2...


# CSV 파일 내보내기

In [ ]:
# csv 파일 내보내기
recipe.to_csv('curry_recipe.csv', encoding='utf-8-sig')